## SDSS Astroquery Batch

Batch runs through the images used in the GZ1 (SDSS DR7) dataset and returns half-light radii etc for data cuts

In [33]:
import numpy as np
import pandas as pd
import time
import math
import os
import random
import astropy.units as u
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord

In [34]:
CATALOG_PATH = '../Data/gz1_desi_cross_cat.csv'
OUTPUT_PATH = '../Data/gz1_desi_cross_cat_queried2.csv'
RADIUS = "1 arcsec"
catalog = pd.read_csv(CATALOG_PATH)

In [40]:
def split_dataframe(data, no_of_batches):
    batch_size = math.ceil(data.shape[0] / no_of_batches)
    batched_df = [data[i:i+batch_size] for i in range(0,data.shape[0], batch_size)]
    return batched_df




def get_SDSS_info_batch():
    batched_df = split_dataframe(catalog,2000) #30s per batch, more than this seems to fail

    if os.path.exists(OUTPUT_PATH):
        os.remove(OUTPUT_PATH)

    for i, batch in enumerate(batched_df):
        print(f"Processing batch {i}")
        
        coords = SkyCoord(batch["RA"],batch["DEC"],unit=(u.hourangle, u.deg))

        results = SDSS.query_region(coords,data_release=7,radius=RADIUS,photoobj_fields=["objID","ra","dec","err_r","petroR50_r","petroR50Err_r"],spectro=True).to_pandas()
        #print(results)
        

        ##THERE IS SOMETHING WRONG WITH THIS CODE - produces 2.6 times the amount of data given??????
        print(batch['OBJID'][0])
        print(results['objID'][0])
        print(batch['OBJID'][0]==results['objID'][0])
        batch['OBJID'] = batch['OBJID'].astype(str).str.strip()
        results['objID'] = results['objID'].astype(str).str.strip()
        #print(results['objID'][0])
        merged_batch = pd.merge(batch,results, how='inner', left_on='OBJID', right_on='objID') # this doesn't work

        # Filter out extra objects
        filtered_df = merged_batch[(merged_batch['OBJID'] != merged_batch['OBJID'].shift(-1))]

        # Remove the last row if it's an extra object
        if merged_batch.iloc[-1]['OBJID'] != merged_batch.iloc[-1]['OBJID']:
            filtered_df = filtered_df[:-1]


        time.sleep(1)
        print(len(batch))
        print(len(results)) # this gives more results than batch?
        print(len(merged_batch))
        print(len(filtered_df))


        #not_in_df1 = [objid for objid in results['objID'] if objid not in batch['OBJID']]

        #print(not_in_df1)
        results.to_csv("test.csv")
        batch.to_csv("test2.csv")
        merged_batch.to_csv(OUTPUT_PATH, mode='a', header=not os.path.exists(OUTPUT_PATH),index=False)

        break


get_SDSS_info_batch()

Processing batch 0


C:\Users\Neil Power\AppData\Local\Temp\ipykernel_17192\3842831564.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['OBJID'] = batch['OBJID'].astype(str).str.strip()


587727177912680556
587727177912680556
True
324
325
325
324


In [36]:
#reduced =  catalog.drop_duplicates(subset=['OBJID'])
reduced = catalog.drop(["Unnamed: 0","Unnamed: 0.1"],axis=1)
print(reduced.shape[0])
print(f"Number of galaxies in GZ1 catalogue: {catalog.shape[0]}")
print(f"Columns: {catalog.columns.values}")
reduced.head(10)
#reduced.to_csv(OUTPUT_PATH,index=False)

KeyError: "['Unnamed: 0.1'] not found in axis"